In [154]:
import os
import glob
import re
from collections import defaultdict
import dask.dataframe as dask_df
import pandas as pd
import numpy as np

# Get and compute csv
***S02_2024 / S01_2024 ... / S01_2019 / S02_2019***

## 01- Generate path files dictionary

In [155]:
# Ruta a la carpeta que contiene los archivos CSV
folder_path = "Data_original/"  # ← cambia esto

# 1. Extraer nombres de fichero .csv
all_files = glob.glob(os.path.join(folder_path, "*.csv"))
file_names = [os.path.basename(f) for f in all_files]

# 2. Función para extraer año y mes
def extract_year_month(filename):
    match = re.search(r"(\d{4})_(\d{2})_", filename)
    if match:
        year = int(match.group(1))
        month = int(match.group(2))
        return (year, month)
    return (0, 0)  # fallback

# 3. Ordenar cronológicamente
file_names_sorted = sorted(file_names, key=extract_year_month)

# 4. Crear diccionario por año y trimestre
files_by_year_quarter = defaultdict(lambda: {"Q1": [], "Q2": [], "Q3": [], "Q4": []})

for file in file_names_sorted:
    year, month = extract_year_month(file)
    if year == 0:
        continue  # omitir ficheros mal formateados

    if 1 <= month <= 3:
        quarter = "Q1"
    elif 4 <= month <= 6:
        quarter = "Q2"
    elif 7 <= month <= 9:
        quarter = "Q3"
    else:
        quarter = "Q4"

    files_by_year_quarter[year][quarter].append(file)

In [156]:
print("Ficheros Q4 2024:", files_by_year_quarter[2024]["Q4"])
print("Ficheros Q3 2024:", files_by_year_quarter[2024]["Q3"])
print("Ficheros Q2 2024:", files_by_year_quarter[2024]["Q2"])
print("Ficheros Q1 2024:", files_by_year_quarter[2024]["Q1"])
print("Ficheros Q4 2023:", files_by_year_quarter[2023]["Q4"])
print("Ficheros Q3 2023:", files_by_year_quarter[2023]["Q3"])
print("Ficheros Q2 2023:", files_by_year_quarter[2023]["Q2"])
print("Ficheros Q1 2023:", files_by_year_quarter[2023]["Q1"])
print("Ficheros Q4 2022:", files_by_year_quarter[2022]["Q4"])
print("Ficheros Q3 2022:", files_by_year_quarter[2022]["Q3"])
print("Ficheros Q2 2022:", files_by_year_quarter[2022]["Q2"])
print("Ficheros Q1 2022:", files_by_year_quarter[2022]["Q1"])
print("Ficheros Q4 2021:", files_by_year_quarter[2021]["Q4"])
print("Ficheros Q3 2021:", files_by_year_quarter[2021]["Q3"])
print("Ficheros Q2 2021:", files_by_year_quarter[2021]["Q2"])
print("Ficheros Q1 2021:", files_by_year_quarter[2021]["Q1"])
print("Ficheros Q4 2020:", files_by_year_quarter[2020]["Q4"])
print("Ficheros Q3 2020:", files_by_year_quarter[2020]["Q3"])
print("Ficheros Q2 2020:", files_by_year_quarter[2020]["Q2"])
print("Ficheros Q1 2020:", files_by_year_quarter[2020]["Q1"])
#print("Ficheros Q4 2019:", files_by_year_quarter[2019]["Q4"])
#print("Ficheros Q3 2019:", files_by_year_quarter[2019]["Q3"])
#print("Ficheros Q2 2019:", files_by_year_quarter[2019]["Q2"])
#print("Ficheros Q1 2019:", files_by_year_quarter[2019]["Q1"])

Ficheros Q4 2024: ['2024_10_Octubre_BicingNou_ESTACIONS.csv', '2024_11_Novembre_BicingNou_ESTACIONS.csv', '2024_12_Desembre_BicingNou_ESTACIONS.csv']
Ficheros Q3 2024: ['2024_07_Juliol_BicingNou_ESTACIONS.csv', '2024_08_Agost_BicingNou_ESTACIONS.csv', '2024_09_Setembre_BicingNou_ESTACIONS.csv']
Ficheros Q2 2024: ['2024_04_Abril_BicingNou_ESTACIONS.csv', '2024_05_Maig_BicingNou_ESTACIONS.csv', '2024_06_Juny_BicingNou_ESTACIONS.csv']
Ficheros Q1 2024: ['2024_01_Gener_BicingNou_ESTACIONS.csv', '2024_02_Febrer_BicingNou_ESTACIONS.csv', '2024_03_Marc_BicingNou_ESTACIONS.csv']
Ficheros Q4 2023: ['2023_10_Octubre_BicingNou_ESTACIONS.csv', '2023_11_Novembre_BicingNou_ESTACIONS.csv', '2023_12_Desembre_BicingNou_ESTACIONS.csv']
Ficheros Q3 2023: ['2023_07_Juliol_BicingNou_ESTACIONS.csv', '2023_08_Agost_BicingNou_ESTACIONS.csv', '2023_09_Setembre_BicingNou_ESTACIONS.csv']
Ficheros Q2 2023: ['2023_04_Abril_BicingNou_ESTACIONS.csv', '2023_05_Maig_BicingNou_ESTACIONS.csv', '2023_06_Juny_BicingNou_ES

## 02- Compute all years registers

### Define dtypes

In [157]:
'''dtypes = {
    'station_id': 'int64',
    'num_bikes_available': 'int64',
    'num_bikes_available_types.mechanical': 'int64',
    'num_bikes_available_types.ebike': 'int64',
    'num_docks_available': 'int64',
    'is_installed': 'int64',
    'is_renting': 'int64',
    'is_returning': 'int64',
    'last_reported': 'float64',
    'is_charging_station': 'bool',
    'status': 'object',
    'last_updated': 'int64',
    'ttl': 'int64'
}'''
dtypes = {
    'station_id': 'object',
    'num_bikes_available': 'object',
    'num_bikes_available_types.mechanical': 'object',
    'num_bikes_available_types.ebike': 'object',
    'num_docks_available': 'object',
    'is_installed': 'object',
    'is_renting': 'object',
    'is_returning': 'object',
    'last_reported': 'object',
    'is_charging_station': 'object',
    'status': 'object',
    'last_updated': 'object',
    'ttl': 'object'
}

### Function "Compute ddf_quarter"

In [158]:
def process_ddf_quarter(ddf):
    # 1. Limpieza y conversión
    ddf['station_id'] = dask_df.to_numeric(ddf['station_id'], errors='coerce')
    ddf = ddf[~ddf['station_id'].isna()]
    ddf['station_id'] = ddf['station_id'].astype('int64')

    ddf['num_docks_available'] = dask_df.to_numeric(ddf['num_docks_available'], errors='coerce')
    ddf['last_reported_dt'] = dask_df.to_datetime(ddf['last_reported'], unit='s')
    ddf['hour_dt'] = ddf['last_reported_dt'].dt.floor('h')

    # 2. Agregación por hora y estación
    ddf_hourly = ddf.groupby(['station_id', 'hour_dt'])[['num_docks_available']].mean().reset_index()

    # 3. Columnas temporales
    ddf_hourly['year'] = ddf_hourly['hour_dt'].dt.year
    ddf_hourly['month'] = ddf_hourly['hour_dt'].dt.month
    ddf_hourly['day'] = ddf_hourly['hour_dt'].dt.day
    ddf_hourly['hour'] = ddf_hourly['hour_dt'].dt.hour

    # 4. Meta
    RETURN_COLUMNS = [
        'station_id', 'hour_dt', 'num_docks_available',
        'year', 'month', 'day', 'hour',
        'ctx_4', 'ctx_3', 'ctx_2', 'ctx_1',
        'row_idx'
    ]

    meta = {
        'station_id': 'int64',
        'hour_dt': 'datetime64[ns]',
        'num_docks_available': 'float64',
        'year': 'float64',
        'month': 'float64',
        'day': 'float64',
        'hour': 'float64',
        'ctx_4': 'float64',
        'ctx_3': 'float64',
        'ctx_2': 'float64',
        'ctx_1': 'float64',
        'row_idx': 'int64'
    }

    # 5. Contexto
    def add_context_and_index(df):
        df = df.sort_values('hour_dt').copy()
        df['ctx_4'] = df['num_docks_available'].shift(4)
        df['ctx_3'] = df['num_docks_available'].shift(3)
        df['ctx_2'] = df['num_docks_available'].shift(2)
        df['ctx_1'] = df['num_docks_available'].shift(1)
        df['row_idx'] = np.arange(len(df))
        return df[RETURN_COLUMNS]

    ddf_with_ctx = ddf_hourly.groupby('station_id').apply(add_context_and_index, meta=meta)

    # 6. Orden y selección
    ddf_final_all = ddf_with_ctx.reset_index(drop=True).map_partitions(
        lambda df: df.sort_values(['station_id', 'hour_dt'])
    )
    ddf_final_subset = ddf_final_all[ddf_final_all['row_idx'] % 5 == 4]

    return ddf_final_subset

### Function "Load files into ddf_quarter"

In [159]:
# Carpeta donde están los CSV
data_folder = "Data_original/"  # ← modifica si es otra

# Función para cargar un quarter completo como Dask DataFrame
def load_quarter(year, quarter, files_by_year_quarter, folder_path=data_folder):
    file_list = files_by_year_quarter.get(year, {}).get(quarter, [])
    full_paths = [os.path.join(folder_path, f) for f in file_list]
    if not full_paths:
        print(f"No files found for {year} {quarter}")
        return None
    return dask_df.read_csv(full_paths, dtype=dtypes, assume_missing=True, include_path_column='source_file')
#    return dask_df.read_csv(full_paths, assume_missing=True)  # Puedes añadir `dtype=...` si sabes los tipos


In [160]:
df_estaciones_pd = pd.read_csv("Informacio_Estacions_Bicing_2025.csv", usecols=["station_id"])
valid_station_ids = df_estaciones_pd["station_id"].dropna().astype("int64").unique().tolist()

### Processing 2024

In [161]:
ddf_2024_Q1 = load_quarter(2024, "Q1", files_by_year_quarter)
ddf_final_subset_2024_q1 = process_ddf_quarter(ddf_2024_Q1)

ddf_2024_Q2 = load_quarter(2024, "Q2", files_by_year_quarter)
ddf_final_subset_2024_q2 = process_ddf_quarter(ddf_2024_Q2)

ddf_2024_Q3 = load_quarter(2024, "Q3", files_by_year_quarter)
ddf_final_subset_2024_q3 = process_ddf_quarter(ddf_2024_Q3)

ddf_2024_Q4 = load_quarter(2024, "Q4", files_by_year_quarter)
ddf_final_subset_2024_q4 = process_ddf_quarter(ddf_2024_Q4)

####  Cecking... NO-filtered data processed

In [162]:
print(len(ddf_final_subset_2024_q1))
ddf_final_subset_2024_q1.head(10)

219458


,station_id,hour_dt,num_docks_available,year,month,day,hour,ctx_4,ctx_3,ctx_2,ctx_1,row_idx
4,2,2024-01-01 02:00:00,15.666667,2024,1,1,2,15.0,13.25,9.916667,8.0,4
9,2,2024-01-01 07:00:00,15.416667,2024,1,1,7,14.5,14.833333,15.384615,16.545455,9
14,2,2024-01-01 12:00:00,16.666667,2024,1,1,12,15.0,16.833333,18.0,17.666667,14
19,2,2024-01-01 17:00:00,15.833333,2024,1,1,17,16.461538,16.818182,16.666667,16.416667,19
24,2,2024-01-01 22:00:00,19.090909,2024,1,1,22,17.333333,17.333333,18.833333,19.769231,24
29,2,2024-01-02 03:00:00,18.0,2024,1,2,3,20.0,20.0,19.833333,18.0,29
34,2,2024-01-02 08:00:00,13.5,2024,1,2,8,18.0,18.0,13.75,14.916667,34
39,2,2024-01-02 13:00:00,22.583333,2024,1,2,13,15.5,18.333333,22.0,21.5,39
44,2,2024-01-02 18:00:00,6.5,2024,1,2,18,23.5,21.583333,22.083333,17.333333,44
49,2,2024-01-02 23:00:00,16.083333,2024,1,2,23,6.666667,7.166667,10.333333,12.583333,49


In [163]:
print(len(ddf_final_subset_2024_q2))
ddf_final_subset_2024_q2.head(10)

KeyboardInterrupt: 

In [ ]:
print(len(ddf_final_subset_2024_q3))
ddf_final_subset_2024_q3.head(10)

In [ ]:
print(len(ddf_final_subset_2024_q4))
ddf_final_subset_2024_q4.head(10)

#### Cecking... Filtered data processed

In [105]:
ddf_final_subset_2024_q1_filtered  = ddf_final_subset_2024_q1[ddf_final_subset_2024_q1['station_id'].isin(valid_station_ids)]
print(len(ddf_final_subset_2024_q1_filtered))
ddf_final_subset_2024_q1_filtered.head(10)

217306


,station_id,hour_dt,num_docks_available,year,month,day,hour,ctx_4,ctx_3,ctx_2,ctx_1,row_idx
4,2,2024-01-01 02:00:00,15.666667,2024,1,1,2,15.000000,13.250000,9.916667,8.000000,4
9,2,2024-01-01 07:00:00,15.416667,2024,1,1,7,14.500000,14.833333,15.384615,16.545455,9
14,2,2024-01-01 12:00:00,16.666667,2024,1,1,12,15.000000,16.833333,18.000000,17.666667,14
19,2,2024-01-01 17:00:00,15.833333,2024,1,1,17,16.461538,16.818182,16.666667,16.416667,19
24,2,2024-01-01 22:00:00,19.090909,2024,1,1,22,17.333333,17.333333,18.833333,19.769231,24
29,2,2024-01-02 03:00:00,18.000000,2024,1,2,3,20.000000,20.000000,19.833333,18.000000,29
34,2,2024-01-02 08:00:00,13.500000,2024,1,2,8,18.000000,18.000000,13.750000,14.916667,34
39,2,2024-01-02 13:00:00,22.583333,2024,1,2,13,15.500000,18.333333,22.000000,21.500000,39
44,2,2024-01-02 18:00:00,6.500000,2024,1,2,18,23.500000,21.583333,22.083333,17.333333,44
49,2,2024-01-02 23:00:00,16.083333,2024,1,2,23,6.666667,7.166667,10.333333,12.583333,49


In [106]:
ddf_final_subset_2024_q2_filtered  = ddf_final_subset_2024_q2[ddf_final_subset_2024_q2['station_id'].isin(valid_station_ids)]
print(len(ddf_final_subset_2024_q2_filtered))
ddf_final_subset_2024_q2_filtered.head(10)

220479


,station_id,hour_dt,num_docks_available,year,month,day,hour,ctx_4,ctx_3,ctx_2,ctx_1,row_idx
4,2,2024-04-01 01:00:00,19.000000,2024.0,4.0,1.0,1.0,19.000000,17.833333,16.666667,18.666667,4
9,2,2024-04-01 06:00:00,7.666667,2024.0,4.0,1.0,6.0,19.000000,10.750000,8.000000,8.250000,9
14,2,2024-04-01 11:00:00,8.916667,2024.0,4.0,1.0,11.0,8.833333,9.416667,11.416667,10.250000,14
19,2,2024-04-01 16:00:00,10.083333,2024.0,4.0,1.0,16.0,6.666667,5.500000,5.416667,7.666667,19
24,2,2024-04-01 21:00:00,5.000000,2024.0,4.0,1.0,21.0,8.083333,4.250000,2.583333,3.250000,24
29,2,2024-04-02 02:00:00,5.000000,2024.0,4.0,2.0,2.0,5.166667,5.000000,5.000000,5.000000,29
34,2,2024-04-02 07:00:00,20.750000,2024.0,4.0,2.0,7.0,5.000000,5.000000,3.333333,11.500000,34
39,2,2024-04-02 12:00:00,20.500000,2024.0,4.0,2.0,12.0,27.000000,20.750000,11.363636,13.916667,39
44,2,2024-04-02 17:00:00,22.000000,2024.0,4.0,2.0,17.0,21.666667,23.583333,21.583333,21.333333,44
49,2,2024-04-02 22:00:00,8.750000,2024.0,4.0,2.0,22.0,7.666667,7.000000,6.500000,8.500000,49


In [107]:
ddf_final_subset_2024_q3_filtered  = ddf_final_subset_2024_q3[ddf_final_subset_2024_q3['station_id'].isin(valid_station_ids)]
print(len(ddf_final_subset_2024_q3_filtered))
ddf_final_subset_2024_q3_filtered.head(10)

222820


,station_id,hour_dt,num_docks_available,year,month,day,hour,ctx_4,ctx_3,ctx_2,ctx_1,row_idx
4,2,2024-07-01 01:00:00,12.916667,2024,7,1,1,18.000000,17.416667,16.454545,15.166667,4
9,2,2024-07-01 06:00:00,11.416667,2024,7,1,6,12.000000,12.000000,13.416667,13.166667,9
14,2,2024-07-01 11:00:00,13.916667,2024,7,1,11,12.833333,13.000000,13.416667,14.666667,14
19,2,2024-07-01 16:00:00,20.333333,2024,7,1,16,14.583333,11.250000,12.583333,15.000000,19
24,2,2024-07-01 21:00:00,24.250000,2024,7,1,21,22.250000,17.666667,16.000000,24.666667,24
29,2,2024-07-02 02:00:00,23.000000,2024,7,2,2,24.750000,23.166667,24.750000,24.000000,29
34,2,2024-07-02 07:00:00,18.666667,2024,7,2,7,23.750000,24.500000,20.000000,26.166667,34
39,2,2024-07-02 12:00:00,17.666667,2024,7,2,12,18.500000,13.916667,14.500000,15.916667,39
44,2,2024-07-02 17:00:00,21.916667,2024,7,2,17,15.083333,18.833333,15.083333,13.166667,44
49,2,2024-07-02 22:00:00,21.666667,2024,7,2,22,25.583333,23.666667,24.083333,24.583333,49


In [108]:
ddf_final_subset_2024_q4_filtered  = ddf_final_subset_2024_q4[ddf_final_subset_2024_q4['station_id'].isin(valid_station_ids)]
print(len(ddf_final_subset_2024_q4_filtered))
ddf_final_subset_2024_q4_filtered.head(10)

222913


,station_id,hour_dt,num_docks_available,year,month,day,hour,ctx_4,ctx_3,ctx_2,ctx_1,row_idx
4,2,2024-10-01 01:00:00,13.583333,2024.0,10.0,1.0,1.0,12.000000,13.750000,14.083333,14.000000,4
9,2,2024-10-01 06:00:00,25.333333,2024.0,10.0,1.0,6.0,14.000000,15.000000,15.250000,12.666667,9
14,2,2024-10-01 11:00:00,21.916667,2024.0,10.0,1.0,11.0,25.250000,18.583333,14.666667,16.416667,14
19,2,2024-10-01 16:00:00,14.833333,2024.0,10.0,1.0,16.0,23.583333,25.166667,23.166667,18.833333,19
24,2,2024-10-01 21:00:00,8.333333,2024.0,10.0,1.0,21.0,18.166667,20.916667,12.583333,10.750000,24
29,2,2024-10-02 02:00:00,6.000000,2024.0,10.0,2.0,2.0,7.916667,7.166667,6.083333,6.000000,29
34,2,2024-10-02 07:00:00,14.500000,2024.0,10.0,2.0,7.0,6.416667,6.833333,4.166667,12.916667,34
39,2,2024-10-02 12:00:00,5.916667,2024.0,10.0,2.0,12.0,19.000000,20.666667,23.333333,8.500000,39
44,2,2024-10-02 17:00:00,5.083333,2024.0,10.0,2.0,17.0,8.166667,12.083333,9.833333,8.333333,44
49,2,2024-10-02 22:00:00,1.750000,2024.0,10.0,2.0,22.0,4.833333,4.500000,4.833333,4.000000,49


### Processing 2023

In [114]:
ddf_2023_Q1 = load_quarter(2023, "Q1", files_by_year_quarter)
ddf_final_subset_2023_q1 = process_ddf_quarter(ddf_2023_Q1)

ddf_2023_Q2 = load_quarter(2023, "Q2", files_by_year_quarter)
ddf_final_subset_2023_q2 = process_ddf_quarter(ddf_2023_Q2)

ddf_2023_Q3 = load_quarter(2023, "Q3", files_by_year_quarter)
ddf_final_subset_2023_q3 = process_ddf_quarter(ddf_2023_Q3)

ddf_2023_Q4 = load_quarter(2023, "Q4", files_by_year_quarter)
ddf_final_subset_2023_q4 = process_ddf_quarter(ddf_2023_Q4)

####  Cecking... NO-filtered data processed

In [115]:
print(len(ddf_final_subset_2023_q1))
ddf_final_subset_2023_q1.head(10)

207204


,station_id,hour_dt,num_docks_available,year,month,day,hour,ctx_4,ctx_3,ctx_2,ctx_1,row_idx
4,2,2023-01-01 02:00:00,17.666667,2023,1,1,2,23.000000,22.833333,21.666667,19.583333,4
9,2,2023-01-01 07:00:00,18.000000,2023,1,1,7,18.333333,18.166667,18.250000,18.000000,9
14,2,2023-01-01 12:00:00,19.083333,2023,1,1,12,17.750000,17.000000,18.333333,18.916667,14
19,2,2023-01-01 17:00:00,21.083333,2023,1,1,17,15.583333,12.833333,14.750000,18.416667,19
24,2,2023-01-01 22:00:00,15.583333,2023,1,1,22,20.583333,21.833333,18.750000,16.500000,24
29,2,2023-01-02 03:00:00,19.000000,2023,1,2,3,17.250000,16.916667,18.000000,18.833333,29
34,2,2023-01-02 08:00:00,20.333333,2023,1,2,8,19.333333,20.250000,19.916667,15.916667,34
39,2,2023-01-02 13:00:00,25.333333,2023,1,2,13,24.583333,25.916667,25.250000,27.166667,39
44,2,2023-01-02 18:00:00,23.750000,2023,1,2,18,23.666667,21.250000,20.083333,22.500000,44
49,2,2023-01-02 23:00:00,23.500000,2023,1,2,23,23.166667,20.833333,21.083333,23.833333,49


In [116]:
print(len(ddf_final_subset_2023_q2))
ddf_final_subset_2023_q2.head(10)

202354


,station_id,hour_dt,num_docks_available,year,month,day,hour,ctx_4,ctx_3,ctx_2,ctx_1,row_idx
4,2,2023-04-01 01:00:00,6.166667,2023,4,1,1,9.000000,6.416667,8.000000,8.000000,4
9,2,2023-04-01 06:00:00,4.666667,2023,4,1,6,3.583333,4.000000,3.416667,4.166667,9
14,2,2023-04-01 11:00:00,10.333333,2023,4,1,11,6.833333,8.083333,5.666667,9.083333,14
19,2,2023-04-01 16:00:00,16.166667,2023,4,1,16,10.833333,12.416667,15.583333,13.833333,19
24,2,2023-04-03 12:00:00,22.000000,2023,4,3,12,19.916667,24.727273,22.000000,21.750000,24
29,2,2023-04-03 17:00:00,24.666667,2023,4,3,17,20.250000,18.500000,23.250000,23.750000,29
34,2,2023-04-03 22:00:00,23.916667,2023,4,3,22,25.416667,25.333333,25.000000,24.333333,34
39,2,2023-04-04 03:00:00,13.333333,2023,4,4,3,23.166667,23.250000,23.250000,13.000000,39
44,2,2023-04-04 08:00:00,20.833333,2023,4,4,8,14.583333,14.250000,19.250000,20.166667,44
49,2,2023-04-04 13:00:00,22.083333,2023,4,4,13,23.333333,22.916667,23.666667,24.500000,49


In [117]:
print(len(ddf_final_subset_2023_q3))
ddf_final_subset_2023_q3.head(10)

155840


,station_id,hour_dt,num_docks_available,year,month,day,hour,ctx_4,ctx_3,ctx_2,ctx_1,row_idx
5,1,2023-07-01 00:00:00,17.166667,2023,7,1,0,1.692140e+09,24.000000,22.166667,21.416667,4
10,1,2023-07-01 05:00:00,26.916667,2023,7,1,5,1.416667e+01,17.416667,21.250000,22.000000,9
15,1,2023-07-01 10:00:00,42.166667,2023,7,1,10,3.133333e+01,34.666667,38.083333,41.166667,14
20,1,2023-07-01 15:00:00,41.000000,2023,7,1,15,4.233333e+01,42.583333,41.416667,42.750000,19
25,1,2023-07-01 20:00:00,38.500000,2023,7,1,20,4.200000e+01,40.333333,39.333333,40.416667,24
30,1,2023-07-02 01:00:00,28.000000,2023,7,2,1,3.933333e+01,34.666667,33.666667,26.916667,29
35,1,2023-07-02 06:00:00,35.833333,2023,7,2,6,2.875000e+01,29.416667,29.916667,32.000000,34
40,1,2023-07-02 11:00:00,39.666667,2023,7,2,11,3.775000e+01,36.000000,33.416667,35.250000,39
45,1,2023-07-02 16:00:00,41.000000,2023,7,2,16,3.875000e+01,39.500000,40.833333,39.500000,44
50,1,2023-07-02 21:00:00,35.000000,2023,7,2,21,3.783333e+01,31.083333,36.416667,36.666667,49


In [118]:
print(len(ddf_final_subset_2023_q4))
ddf_final_subset_2023_q4.head(10)

221692


,station_id,hour_dt,num_docks_available,year,month,day,hour,ctx_4,ctx_3,ctx_2,ctx_1,row_idx
4,2,2023-10-01 01:00:00,9.916667,2023,10,1,1,12.000000,13.916667,12.416667,9.416667,4
9,2,2023-10-01 06:00:00,7.909091,2023,10,1,6,8.666667,8.615385,7.000000,7.307692,9
14,2,2023-10-01 11:00:00,19.230769,2023,10,1,11,7.000000,9.500000,15.846154,18.090909,14
19,2,2023-10-01 16:00:00,26.250000,2023,10,1,16,20.363636,23.692308,23.272727,23.833333,19
24,2,2023-10-01 21:00:00,14.000000,2023,10,1,21,17.000000,12.307692,14.909091,14.750000,24
29,2,2023-10-02 02:00:00,15.000000,2023,10,2,2,14.833333,14.666667,15.076923,15.000000,29
34,2,2023-10-02 07:00:00,24.153846,2023,10,2,7,15.000000,14.909091,12.538462,20.090909,34
39,2,2023-10-02 12:00:00,20.769231,2023,10,2,12,12.818182,17.000000,16.818182,18.333333,39
44,2,2023-10-02 17:00:00,20.666667,2023,10,2,17,18.545455,13.384615,11.727273,14.333333,44
49,2,2023-10-02 22:00:00,24.000000,2023,10,2,22,25.615385,21.818182,24.250000,22.750000,49


#### Cecking... Filtered data processed

In [119]:
ddf_final_subset_2023_q1_filtered = ddf_final_subset_2023_q1[ddf_final_subset_2023_q1['station_id'].isin(valid_station_ids)]
print(len(ddf_final_subset_2023_q1_filtered))
ddf_final_subset_2023_q1_filtered.head(10)

205077


,station_id,hour_dt,num_docks_available,year,month,day,hour,ctx_4,ctx_3,ctx_2,ctx_1,row_idx
4,2,2023-01-01 02:00:00,17.666667,2023,1,1,2,23.000000,22.833333,21.666667,19.583333,4
9,2,2023-01-01 07:00:00,18.000000,2023,1,1,7,18.333333,18.166667,18.250000,18.000000,9
14,2,2023-01-01 12:00:00,19.083333,2023,1,1,12,17.750000,17.000000,18.333333,18.916667,14
19,2,2023-01-01 17:00:00,21.083333,2023,1,1,17,15.583333,12.833333,14.750000,18.416667,19
24,2,2023-01-01 22:00:00,15.583333,2023,1,1,22,20.583333,21.833333,18.750000,16.500000,24
29,2,2023-01-02 03:00:00,19.000000,2023,1,2,3,17.250000,16.916667,18.000000,18.833333,29
34,2,2023-01-02 08:00:00,20.333333,2023,1,2,8,19.333333,20.250000,19.916667,15.916667,34
39,2,2023-01-02 13:00:00,25.333333,2023,1,2,13,24.583333,25.916667,25.250000,27.166667,39
44,2,2023-01-02 18:00:00,23.750000,2023,1,2,18,23.666667,21.250000,20.083333,22.500000,44
49,2,2023-01-02 23:00:00,23.500000,2023,1,2,23,23.166667,20.833333,21.083333,23.833333,49


In [120]:
ddf_final_subset_2023_q2_filtered = ddf_final_subset_2023_q2[ddf_final_subset_2023_q2['station_id'].isin(valid_station_ids)]
print(len(ddf_final_subset_2023_q2_filtered))
ddf_final_subset_2023_q2_filtered.head(10)

200342


,station_id,hour_dt,num_docks_available,year,month,day,hour,ctx_4,ctx_3,ctx_2,ctx_1,row_idx
4,2,2023-04-01 01:00:00,6.166667,2023,4,1,1,9.000000,6.416667,8.000000,8.000000,4
9,2,2023-04-01 06:00:00,4.666667,2023,4,1,6,3.583333,4.000000,3.416667,4.166667,9
14,2,2023-04-01 11:00:00,10.333333,2023,4,1,11,6.833333,8.083333,5.666667,9.083333,14
19,2,2023-04-01 16:00:00,16.166667,2023,4,1,16,10.833333,12.416667,15.583333,13.833333,19
24,2,2023-04-03 12:00:00,22.000000,2023,4,3,12,19.916667,24.727273,22.000000,21.750000,24
29,2,2023-04-03 17:00:00,24.666667,2023,4,3,17,20.250000,18.500000,23.250000,23.750000,29
34,2,2023-04-03 22:00:00,23.916667,2023,4,3,22,25.416667,25.333333,25.000000,24.333333,34
39,2,2023-04-04 03:00:00,13.333333,2023,4,4,3,23.166667,23.250000,23.250000,13.000000,39
44,2,2023-04-04 08:00:00,20.833333,2023,4,4,8,14.583333,14.250000,19.250000,20.166667,44
49,2,2023-04-04 13:00:00,22.083333,2023,4,4,13,23.333333,22.916667,23.666667,24.500000,49


In [121]:
ddf_final_subset_2023_q3_filtered = ddf_final_subset_2023_q3[ddf_final_subset_2023_q3['station_id'].isin(valid_station_ids)]
print(len(ddf_final_subset_2023_q3_filtered))
ddf_final_subset_2023_q3_filtered.head(10)

154328


,station_id,hour_dt,num_docks_available,year,month,day,hour,ctx_4,ctx_3,ctx_2,ctx_1,row_idx
5,1,2023-07-01 00:00:00,17.166667,2023,7,1,0,1.692140e+09,24.000000,22.166667,21.416667,4
10,1,2023-07-01 05:00:00,26.916667,2023,7,1,5,1.416667e+01,17.416667,21.250000,22.000000,9
15,1,2023-07-01 10:00:00,42.166667,2023,7,1,10,3.133333e+01,34.666667,38.083333,41.166667,14
20,1,2023-07-01 15:00:00,41.000000,2023,7,1,15,4.233333e+01,42.583333,41.416667,42.750000,19
25,1,2023-07-01 20:00:00,38.500000,2023,7,1,20,4.200000e+01,40.333333,39.333333,40.416667,24
30,1,2023-07-02 01:00:00,28.000000,2023,7,2,1,3.933333e+01,34.666667,33.666667,26.916667,29
35,1,2023-07-02 06:00:00,35.833333,2023,7,2,6,2.875000e+01,29.416667,29.916667,32.000000,34
40,1,2023-07-02 11:00:00,39.666667,2023,7,2,11,3.775000e+01,36.000000,33.416667,35.250000,39
45,1,2023-07-02 16:00:00,41.000000,2023,7,2,16,3.875000e+01,39.500000,40.833333,39.500000,44
50,1,2023-07-02 21:00:00,35.000000,2023,7,2,21,3.783333e+01,31.083333,36.416667,36.666667,49


In [122]:
ddf_final_subset_2023_q4_filtered = ddf_final_subset_2023_q4[ddf_final_subset_2023_q4['station_id'].isin(valid_station_ids)]
print(len(ddf_final_subset_2023_q4_filtered))
ddf_final_subset_2023_q4_filtered.head(10)

219484


,station_id,hour_dt,num_docks_available,year,month,day,hour,ctx_4,ctx_3,ctx_2,ctx_1,row_idx
4,2,2023-10-01 01:00:00,9.916667,2023,10,1,1,12.000000,13.916667,12.416667,9.416667,4
9,2,2023-10-01 06:00:00,7.909091,2023,10,1,6,8.666667,8.615385,7.000000,7.307692,9
14,2,2023-10-01 11:00:00,19.230769,2023,10,1,11,7.000000,9.500000,15.846154,18.090909,14
19,2,2023-10-01 16:00:00,26.250000,2023,10,1,16,20.363636,23.692308,23.272727,23.833333,19
24,2,2023-10-01 21:00:00,14.000000,2023,10,1,21,17.000000,12.307692,14.909091,14.750000,24
29,2,2023-10-02 02:00:00,15.000000,2023,10,2,2,14.833333,14.666667,15.076923,15.000000,29
34,2,2023-10-02 07:00:00,24.153846,2023,10,2,7,15.000000,14.909091,12.538462,20.090909,34
39,2,2023-10-02 12:00:00,20.769231,2023,10,2,12,12.818182,17.000000,16.818182,18.333333,39
44,2,2023-10-02 17:00:00,20.666667,2023,10,2,17,18.545455,13.384615,11.727273,14.333333,44
49,2,2023-10-02 22:00:00,24.000000,2023,10,2,22,25.615385,21.818182,24.250000,22.750000,49


### Processing 2022

In [111]:
ddf_2022_Q1 = load_quarter(2022, "Q1", files_by_year_quarter)
ddf_final_subset_2022_q1 = process_ddf_quarter(ddf_2022_Q1)

ddf_2022_Q2 = load_quarter(2022, "Q2", files_by_year_quarter)
ddf_final_subset_2022_q2 = process_ddf_quarter(ddf_2022_Q2)

ddf_2022_Q3 = load_quarter(2022, "Q3", files_by_year_quarter)
ddf_final_subset_2022_q3 = process_ddf_quarter(ddf_2022_Q3)

ddf_2022_Q4 = load_quarter(2022, "Q4", files_by_year_quarter)
ddf_final_subset_2022_q4 = process_ddf_quarter(ddf_2022_Q4)

####  Cecking... NO-filtered data processed

In [140]:
print(len(ddf_final_subset_2022_q1))
ddf_final_subset_2022_q1.head(10)

217270


,station_id,hour_dt,num_docks_available,year,month,day,hour,ctx_4,ctx_3,ctx_2,ctx_1,row_idx
4,2,2022-01-01 02:00:00,12.583333,2022,1,1,2,11.000000,10.583333,13.750000,13.000000,4
9,2,2022-01-01 07:00:00,16.416667,2022,1,1,7,12.916667,13.000000,14.250000,16.000000,9
14,2,2022-01-01 12:00:00,18.916667,2022,1,1,12,16.000000,15.416667,14.000000,15.500000,14
19,2,2022-01-01 17:00:00,0.916667,2022,1,1,17,19.166667,15.916667,14.083333,8.666667,19
24,2,2022-01-01 22:00:00,3.000000,2022,1,1,22,1.916667,1.000000,2.000000,4.583333,24
29,2,2022-01-02 03:00:00,5.000000,2022,1,2,3,3.916667,4.500000,5.000000,5.000000,29
34,2,2022-01-02 08:00:00,6.833333,2022,1,2,8,5.000000,5.000000,5.000000,5.250000,34
39,2,2022-01-02 13:00:00,9.333333,2022,1,2,13,5.666667,8.083333,9.750000,9.666667,39
44,2,2022-01-02 18:00:00,12.166667,2022,1,2,18,13.500000,15.166667,13.666667,12.000000,44
49,2,2022-01-02 23:00:00,9.500000,2022,1,2,23,10.833333,8.750000,8.000000,10.333333,49


In [141]:
print(len(ddf_final_subset_2022_q2))
ddf_final_subset_2022_q2.head(10)

221843


,station_id,hour_dt,num_docks_available,year,month,day,hour,ctx_4,ctx_3,ctx_2,ctx_1,row_idx
4,2,2022-04-01 01:00:00,13.000000,2022,4,1,1,14.000000,14.333333,14.166667,14.000000,4
9,2,2022-04-01 06:00:00,13.750000,2022,4,1,6,13.000000,13.000000,13.083333,13.416667,9
14,2,2022-04-01 11:00:00,19.916667,2022,4,1,11,18.500000,18.916667,22.833333,14.333333,14
19,2,2022-04-01 16:00:00,24.833333,2022,4,1,16,21.583333,20.583333,22.416667,24.833333,19
24,2,2022-04-01 21:00:00,14.583333,2022,4,1,21,22.250000,18.166667,17.083333,14.666667,24
29,2,2022-04-02 02:00:00,14.000000,2022,4,2,2,13.250000,12.166667,12.500000,13.166667,29
34,2,2022-04-02 07:00:00,16.750000,2022,4,2,7,14.000000,13.666667,14.000000,14.833333,34
39,2,2022-04-02 12:00:00,14.166667,2022,4,2,12,16.000000,15.666667,14.916667,14.833333,39
44,2,2022-04-02 17:00:00,18.916667,2022,4,2,17,14.333333,15.083333,15.583333,13.750000,44
49,2,2022-04-02 22:00:00,20.250000,2022,4,2,22,21.583333,21.083333,20.750000,19.916667,49


In [142]:
print(len(ddf_final_subset_2022_q3))
ddf_final_subset_2022_q3.head(10)

223242


,station_id,hour_dt,num_docks_available,year,month,day,hour,ctx_4,ctx_3,ctx_2,ctx_1,row_idx
4,2,2022-07-01 01:00:00,12.000000,2022,7,1,1,15.000000,13.500000,13.333333,12.416667,4
9,2,2022-07-01 06:00:00,8.250000,2022,7,1,6,11.916667,12.000000,12.000000,10.833333,9
14,2,2022-07-01 11:00:00,27.083333,2022,7,1,11,13.833333,18.333333,25.250000,26.166667,14
19,2,2022-07-01 16:00:00,25.333333,2022,7,1,16,25.000000,25.416667,27.083333,24.500000,19
24,2,2022-07-01 21:00:00,22.916667,2022,7,1,21,21.833333,18.500000,21.833333,23.750000,24
29,2,2022-07-02 02:00:00,23.916667,2022,7,2,2,20.666667,23.500000,21.500000,19.916667,29
34,2,2022-07-02 07:00:00,26.083333,2022,7,2,7,23.750000,25.666667,26.000000,27.083333,34
39,2,2022-07-02 12:00:00,28.166667,2022,7,2,12,25.250000,26.916667,28.000000,28.000000,39
44,2,2022-07-02 17:00:00,23.000000,2022,7,2,17,25.750000,28.500000,28.083333,25.083333,44
49,2,2022-07-02 22:00:00,21.666667,2022,7,2,22,21.666667,20.250000,20.583333,18.583333,49


In [143]:
print(len(ddf_final_subset_2022_q4))
ddf_final_subset_2022_q4.head(10)

222863


,station_id,hour_dt,num_docks_available,year,month,day,hour,ctx_4,ctx_3,ctx_2,ctx_1,row_idx
4,2,2022-10-01 01:00:00,4.833333,2022,10,1,1,4.000000,3.333333,4.000000,4.583333,4
9,2,2022-10-01 06:00:00,5.000000,2022,10,1,6,6.000000,7.000000,4.250000,4.000000,9
14,2,2022-10-01 11:00:00,18.666667,2022,10,1,11,10.583333,15.333333,14.500000,16.750000,14
19,2,2022-10-01 16:00:00,18.000000,2022,10,1,16,20.500000,21.500000,22.083333,23.416667,19
24,2,2022-10-01 21:00:00,7.833333,2022,10,1,21,13.333333,14.916667,11.416667,9.083333,24
29,2,2022-10-02 02:00:00,7.833333,2022,10,2,2,11.416667,11.000000,9.416667,8.750000,29
34,2,2022-10-02 07:00:00,7.500000,2022,10,2,7,7.666667,8.000000,8.000000,7.500000,34
39,2,2022-10-02 12:00:00,4.000000,2022,10,2,12,7.666667,8.583333,9.750000,6.750000,39
44,2,2022-10-02 17:00:00,18.416667,2022,10,2,17,6.250000,10.833333,14.916667,15.750000,44
49,2,2022-10-02 22:00:00,12.750000,2022,10,2,22,16.000000,15.583333,13.916667,13.166667,49


#### Cecking... Filtered data processed

In [144]:
ddf_final_subset_2022_q1_filtered = ddf_final_subset_2022_q1[ddf_final_subset_2022_q1['station_id'].isin(valid_station_ids)]
print(len(ddf_final_subset_2022_q1_filtered))
ddf_final_subset_2022_q1_filtered.head(10)

215136


,station_id,hour_dt,num_docks_available,year,month,day,hour,ctx_4,ctx_3,ctx_2,ctx_1,row_idx
4,2,2022-01-01 02:00:00,12.583333,2022,1,1,2,11.000000,10.583333,13.750000,13.000000,4
9,2,2022-01-01 07:00:00,16.416667,2022,1,1,7,12.916667,13.000000,14.250000,16.000000,9
14,2,2022-01-01 12:00:00,18.916667,2022,1,1,12,16.000000,15.416667,14.000000,15.500000,14
19,2,2022-01-01 17:00:00,0.916667,2022,1,1,17,19.166667,15.916667,14.083333,8.666667,19
24,2,2022-01-01 22:00:00,3.000000,2022,1,1,22,1.916667,1.000000,2.000000,4.583333,24
29,2,2022-01-02 03:00:00,5.000000,2022,1,2,3,3.916667,4.500000,5.000000,5.000000,29
34,2,2022-01-02 08:00:00,6.833333,2022,1,2,8,5.000000,5.000000,5.000000,5.250000,34
39,2,2022-01-02 13:00:00,9.333333,2022,1,2,13,5.666667,8.083333,9.750000,9.666667,39
44,2,2022-01-02 18:00:00,12.166667,2022,1,2,18,13.500000,15.166667,13.666667,12.000000,44
49,2,2022-01-02 23:00:00,9.500000,2022,1,2,23,10.833333,8.750000,8.000000,10.333333,49


In [145]:
ddf_final_subset_2022_q2_filtered = ddf_final_subset_2022_q2[ddf_final_subset_2022_q2['station_id'].isin(valid_station_ids)]
print(len(ddf_final_subset_2022_q2_filtered))
ddf_final_subset_2022_q2_filtered.head(10)

219674


,station_id,hour_dt,num_docks_available,year,month,day,hour,ctx_4,ctx_3,ctx_2,ctx_1,row_idx
4,2,2022-04-01 01:00:00,13.000000,2022,4,1,1,14.000000,14.333333,14.166667,14.000000,4
9,2,2022-04-01 06:00:00,13.750000,2022,4,1,6,13.000000,13.000000,13.083333,13.416667,9
14,2,2022-04-01 11:00:00,19.916667,2022,4,1,11,18.500000,18.916667,22.833333,14.333333,14
19,2,2022-04-01 16:00:00,24.833333,2022,4,1,16,21.583333,20.583333,22.416667,24.833333,19
24,2,2022-04-01 21:00:00,14.583333,2022,4,1,21,22.250000,18.166667,17.083333,14.666667,24
29,2,2022-04-02 02:00:00,14.000000,2022,4,2,2,13.250000,12.166667,12.500000,13.166667,29
34,2,2022-04-02 07:00:00,16.750000,2022,4,2,7,14.000000,13.666667,14.000000,14.833333,34
39,2,2022-04-02 12:00:00,14.166667,2022,4,2,12,16.000000,15.666667,14.916667,14.833333,39
44,2,2022-04-02 17:00:00,18.916667,2022,4,2,17,14.333333,15.083333,15.583333,13.750000,44
49,2,2022-04-02 22:00:00,20.250000,2022,4,2,22,21.583333,21.083333,20.750000,19.916667,49


In [146]:
ddf_final_subset_2022_q3_filtered = ddf_final_subset_2022_q3[ddf_final_subset_2022_q3['station_id'].isin(valid_station_ids)]
print(len(ddf_final_subset_2022_q3_filtered))
ddf_final_subset_2022_q3_filtered.head(10)

221034


,station_id,hour_dt,num_docks_available,year,month,day,hour,ctx_4,ctx_3,ctx_2,ctx_1,row_idx
4,2,2022-07-01 01:00:00,12.000000,2022,7,1,1,15.000000,13.500000,13.333333,12.416667,4
9,2,2022-07-01 06:00:00,8.250000,2022,7,1,6,11.916667,12.000000,12.000000,10.833333,9
14,2,2022-07-01 11:00:00,27.083333,2022,7,1,11,13.833333,18.333333,25.250000,26.166667,14
19,2,2022-07-01 16:00:00,25.333333,2022,7,1,16,25.000000,25.416667,27.083333,24.500000,19
24,2,2022-07-01 21:00:00,22.916667,2022,7,1,21,21.833333,18.500000,21.833333,23.750000,24
29,2,2022-07-02 02:00:00,23.916667,2022,7,2,2,20.666667,23.500000,21.500000,19.916667,29
34,2,2022-07-02 07:00:00,26.083333,2022,7,2,7,23.750000,25.666667,26.000000,27.083333,34
39,2,2022-07-02 12:00:00,28.166667,2022,7,2,12,25.250000,26.916667,28.000000,28.000000,39
44,2,2022-07-02 17:00:00,23.000000,2022,7,2,17,25.750000,28.500000,28.083333,25.083333,44
49,2,2022-07-02 22:00:00,21.666667,2022,7,2,22,21.666667,20.250000,20.583333,18.583333,49


In [147]:
ddf_final_subset_2022_q4_filtered = ddf_final_subset_2022_q4[ddf_final_subset_2022_q4['station_id'].isin(valid_station_ids)]
print(len(ddf_final_subset_2022_q4_filtered))
ddf_final_subset_2022_q4_filtered.head(10)

220644


,station_id,hour_dt,num_docks_available,year,month,day,hour,ctx_4,ctx_3,ctx_2,ctx_1,row_idx
4,2,2022-10-01 01:00:00,4.833333,2022,10,1,1,4.000000,3.333333,4.000000,4.583333,4
9,2,2022-10-01 06:00:00,5.000000,2022,10,1,6,6.000000,7.000000,4.250000,4.000000,9
14,2,2022-10-01 11:00:00,18.666667,2022,10,1,11,10.583333,15.333333,14.500000,16.750000,14
19,2,2022-10-01 16:00:00,18.000000,2022,10,1,16,20.500000,21.500000,22.083333,23.416667,19
24,2,2022-10-01 21:00:00,7.833333,2022,10,1,21,13.333333,14.916667,11.416667,9.083333,24
29,2,2022-10-02 02:00:00,7.833333,2022,10,2,2,11.416667,11.000000,9.416667,8.750000,29
34,2,2022-10-02 07:00:00,7.500000,2022,10,2,7,7.666667,8.000000,8.000000,7.500000,34
39,2,2022-10-02 12:00:00,4.000000,2022,10,2,12,7.666667,8.583333,9.750000,6.750000,39
44,2,2022-10-02 17:00:00,18.416667,2022,10,2,17,6.250000,10.833333,14.916667,15.750000,44
49,2,2022-10-02 22:00:00,12.750000,2022,10,2,22,16.000000,15.583333,13.916667,13.166667,49


### Processing 2021

In [112]:
ddf_2021_Q1 = load_quarter(2021, "Q1", files_by_year_quarter)
ddf_final_subset_2021_q1 = process_ddf_quarter(ddf_2021_Q1)

ddf_2021_Q2 = load_quarter(2021, "Q2", files_by_year_quarter)
ddf_final_subset_2021_q2 = process_ddf_quarter(ddf_2021_Q2)

ddf_2021_Q3 = load_quarter(2021, "Q3", files_by_year_quarter)
ddf_final_subset_2021_q3 = process_ddf_quarter(ddf_2021_Q3)

ddf_2021_Q4 = load_quarter(2021, "Q4", files_by_year_quarter)
ddf_final_subset_2021_q4 = process_ddf_quarter(ddf_2021_Q4)

####  Cecking... NO-filtered data processed

In [132]:
print(len(ddf_final_subset_2021_q1))
ddf_final_subset_2021_q1.head(10)

217464


,station_id,hour_dt,num_docks_available,year,month,day,hour,ctx_4,ctx_3,ctx_2,ctx_1,row_idx
4,2,2021-01-01 02:00:00,14.000000,2021,1,1,2,10.000000,12.818182,13.230769,13.000000,4
9,2,2021-01-01 07:00:00,14.153846,2021,1,1,7,15.833333,16.000000,15.166667,14.363636,9
14,2,2021-01-01 12:00:00,16.545455,2021,1,1,12,15.000000,15.166667,15.583333,16.000000,14
19,2,2021-01-01 17:00:00,19.250000,2021,1,1,17,19.923077,20.666667,20.416667,17.750000,19
24,2,2021-01-01 22:00:00,14.333333,2021,1,1,22,19.833333,17.750000,16.916667,14.916667,24
29,2,2021-01-02 03:00:00,14.000000,2021,1,2,3,14.000000,14.000000,14.000000,14.000000,29
34,2,2021-01-02 08:00:00,14.307692,2021,1,2,8,14.000000,14.000000,14.153846,14.909091,34
39,2,2021-01-02 13:00:00,24.333333,2021,1,2,13,14.416667,18.333333,19.916667,23.000000,39
44,2,2021-01-02 18:00:00,16.583333,2021,1,2,18,24.666667,20.583333,21.333333,18.583333,44
49,2,2021-01-02 23:00:00,13.000000,2021,1,2,23,16.333333,15.583333,12.916667,13.833333,49


In [133]:
print(len(ddf_final_subset_2021_q2))
ddf_final_subset_2021_q2.head(10)

219975


,station_id,hour_dt,num_docks_available,year,month,day,hour,ctx_4,ctx_3,ctx_2,ctx_1,row_idx
4,2,2021-04-01 02:00:00,18.000000,2021,4,1,2,17.000000,18.000000,18.000000,18.000000,4
9,2,2021-04-01 07:00:00,18.250000,2021,4,1,7,18.000000,18.000000,18.333333,16.166667,9
14,2,2021-04-01 12:00:00,24.230769,2021,4,1,12,21.250000,23.000000,25.750000,23.727273,14
19,2,2021-04-01 17:00:00,21.333333,2021,4,1,17,25.250000,23.583333,24.583333,24.666667,19
24,2,2021-04-01 22:00:00,24.000000,2021,4,1,22,19.363636,22.230769,24.833333,24.000000,24
29,2,2021-04-02 10:00:00,25.545455,2021,4,2,10,24.000000,24.000000,25.000000,25.750000,29
34,2,2021-04-02 16:00:00,16.888889,2021,4,2,16,23.615385,22.666667,21.916667,21.777778,34
39,2,2021-04-02 21:00:00,4.500000,2021,4,2,21,15.583333,13.750000,10.000000,4.666667,39
44,2,2021-04-03 02:00:00,2.000000,2021,4,3,2,2.000000,2.000000,2.000000,2.000000,44
49,2,2021-04-03 07:00:00,5.250000,2021,4,3,7,2.000000,2.000000,2.461538,3.416667,49


In [134]:
print(len(ddf_final_subset_2021_q3))
ddf_final_subset_2021_q3.head(10)

222015


,station_id,hour_dt,num_docks_available,year,month,day,hour,ctx_4,ctx_3,ctx_2,ctx_1,row_idx
4,2,2021-07-01 01:00:00,11.000000,2021,7,1,1,16.000000,15.083333,11.916667,11.000000,4
9,2,2021-07-01 06:00:00,9.250000,2021,7,1,6,10.416667,10.166667,10.666667,9.500000,9
14,2,2021-07-01 11:00:00,20.833333,2021,7,1,11,18.416667,17.416667,17.583333,23.416667,14
19,2,2021-07-01 16:00:00,25.833333,2021,7,1,16,21.083333,19.166667,17.166667,22.583333,19
24,2,2021-07-01 21:00:00,16.166667,2021,7,1,21,22.250000,23.250000,22.083333,18.833333,24
29,2,2021-07-02 02:00:00,15.000000,2021,7,2,2,15.000000,12.750000,13.166667,14.750000,29
34,2,2021-07-02 07:00:00,16.583333,2021,7,2,7,15.000000,14.833333,16.083333,12.583333,34
39,2,2021-07-02 12:00:00,23.916667,2021,7,2,12,19.083333,22.166667,25.000000,23.083333,39
44,2,2021-07-02 17:00:00,18.916667,2021,7,2,17,26.166667,26.416667,25.916667,23.250000,44
49,2,2021-07-02 22:00:00,10.833333,2021,7,2,22,18.416667,17.500000,15.833333,15.083333,49


In [135]:
print(len(ddf_final_subset_2021_q4))
ddf_final_subset_2021_q4.head(10)

111831


,station_id,hour_dt,num_docks_available,year,month,day,hour,ctx_4,ctx_3,ctx_2,ctx_1,row_idx
4,2,2021-10-01 00:00:00,28.000000,2021,10,1,0,1.638471e+09,24.000000,26.166667,27.583333,4
9,2,2021-10-01 05:00:00,9.083333,2021,10,1,5,2.800000e+01,28.000000,26.333333,10.583333,9
14,2,2021-10-01 10:00:00,24.500000,2021,10,1,10,8.666667e+00,20.500000,26.000000,25.083333,14
19,2,2021-10-01 15:00:00,21.166667,2021,10,1,15,1.983333e+01,8.583333,13.333333,15.916667,19
24,2,2021-10-01 20:00:00,25.416667,2021,10,1,20,2.433333e+01,26.500000,27.416667,25.500000,24
29,2,2021-10-02 01:00:00,27.166667,2021,10,2,1,2.583333e+01,25.416667,24.000000,24.833333,29
34,2,2021-10-02 06:00:00,15.000000,2021,10,2,6,2.600000e+01,24.416667,14.500000,14.166667,34
39,2,2021-10-02 11:00:00,25.083333,2021,10,2,11,1.566667e+01,20.333333,21.666667,23.333333,39
44,2,2021-10-02 16:00:00,24.166667,2021,10,2,16,2.366667e+01,15.250000,22.166667,25.500000,44
49,2,2021-10-02 21:00:00,26.666667,2021,10,2,21,2.466667e+01,26.250000,26.166667,25.416667,49


#### Cecking... Filtered data processed

In [136]:
ddf_final_subset_2021_q1_filtered = ddf_final_subset_2021_q1[ddf_final_subset_2021_q1['station_id'].isin(valid_station_ids)]
print(len(ddf_final_subset_2021_q1_filtered))
ddf_final_subset_2021_q1_filtered.head(10)

215319


,station_id,hour_dt,num_docks_available,year,month,day,hour,ctx_4,ctx_3,ctx_2,ctx_1,row_idx
4,2,2021-01-01 02:00:00,14.000000,2021,1,1,2,10.000000,12.818182,13.230769,13.000000,4
9,2,2021-01-01 07:00:00,14.153846,2021,1,1,7,15.833333,16.000000,15.166667,14.363636,9
14,2,2021-01-01 12:00:00,16.545455,2021,1,1,12,15.000000,15.166667,15.583333,16.000000,14
19,2,2021-01-01 17:00:00,19.250000,2021,1,1,17,19.923077,20.666667,20.416667,17.750000,19
24,2,2021-01-01 22:00:00,14.333333,2021,1,1,22,19.833333,17.750000,16.916667,14.916667,24
29,2,2021-01-02 03:00:00,14.000000,2021,1,2,3,14.000000,14.000000,14.000000,14.000000,29
34,2,2021-01-02 08:00:00,14.307692,2021,1,2,8,14.000000,14.000000,14.153846,14.909091,34
39,2,2021-01-02 13:00:00,24.333333,2021,1,2,13,14.416667,18.333333,19.916667,23.000000,39
44,2,2021-01-02 18:00:00,16.583333,2021,1,2,18,24.666667,20.583333,21.333333,18.583333,44
49,2,2021-01-02 23:00:00,13.000000,2021,1,2,23,16.333333,15.583333,12.916667,13.833333,49


In [137]:
ddf_final_subset_2021_q2_filtered = ddf_final_subset_2021_q2[ddf_final_subset_2021_q2['station_id'].isin(valid_station_ids)]
print(len(ddf_final_subset_2021_q2_filtered))
ddf_final_subset_2021_q2_filtered.head(10)

217809


,station_id,hour_dt,num_docks_available,year,month,day,hour,ctx_4,ctx_3,ctx_2,ctx_1,row_idx
4,2,2021-04-01 02:00:00,18.000000,2021,4,1,2,17.000000,18.000000,18.000000,18.000000,4
9,2,2021-04-01 07:00:00,18.250000,2021,4,1,7,18.000000,18.000000,18.333333,16.166667,9
14,2,2021-04-01 12:00:00,24.230769,2021,4,1,12,21.250000,23.000000,25.750000,23.727273,14
19,2,2021-04-01 17:00:00,21.333333,2021,4,1,17,25.250000,23.583333,24.583333,24.666667,19
24,2,2021-04-01 22:00:00,24.000000,2021,4,1,22,19.363636,22.230769,24.833333,24.000000,24
29,2,2021-04-02 10:00:00,25.545455,2021,4,2,10,24.000000,24.000000,25.000000,25.750000,29
34,2,2021-04-02 16:00:00,16.888889,2021,4,2,16,23.615385,22.666667,21.916667,21.777778,34
39,2,2021-04-02 21:00:00,4.500000,2021,4,2,21,15.583333,13.750000,10.000000,4.666667,39
44,2,2021-04-03 02:00:00,2.000000,2021,4,3,2,2.000000,2.000000,2.000000,2.000000,44
49,2,2021-04-03 07:00:00,5.250000,2021,4,3,7,2.000000,2.000000,2.461538,3.416667,49


In [138]:
ddf_final_subset_2021_q3_filtered = ddf_final_subset_2021_q3[ddf_final_subset_2021_q3['station_id'].isin(valid_station_ids)]
print(len(ddf_final_subset_2021_q3_filtered))
ddf_final_subset_2021_q3_filtered.head(10)

219823


,station_id,hour_dt,num_docks_available,year,month,day,hour,ctx_4,ctx_3,ctx_2,ctx_1,row_idx
4,2,2021-07-01 01:00:00,11.000000,2021,7,1,1,16.000000,15.083333,11.916667,11.000000,4
9,2,2021-07-01 06:00:00,9.250000,2021,7,1,6,10.416667,10.166667,10.666667,9.500000,9
14,2,2021-07-01 11:00:00,20.833333,2021,7,1,11,18.416667,17.416667,17.583333,23.416667,14
19,2,2021-07-01 16:00:00,25.833333,2021,7,1,16,21.083333,19.166667,17.166667,22.583333,19
24,2,2021-07-01 21:00:00,16.166667,2021,7,1,21,22.250000,23.250000,22.083333,18.833333,24
29,2,2021-07-02 02:00:00,15.000000,2021,7,2,2,15.000000,12.750000,13.166667,14.750000,29
34,2,2021-07-02 07:00:00,16.583333,2021,7,2,7,15.000000,14.833333,16.083333,12.583333,34
39,2,2021-07-02 12:00:00,23.916667,2021,7,2,12,19.083333,22.166667,25.000000,23.083333,39
44,2,2021-07-02 17:00:00,18.916667,2021,7,2,17,26.166667,26.416667,25.916667,23.250000,44
49,2,2021-07-02 22:00:00,10.833333,2021,7,2,22,18.416667,17.500000,15.833333,15.083333,49


In [139]:
ddf_final_subset_2021_q4_filtered = ddf_final_subset_2021_q4[ddf_final_subset_2021_q4['station_id'].isin(valid_station_ids)]
print(len(ddf_final_subset_2021_q4_filtered))
ddf_final_subset_2021_q4_filtered.head(10)

110726


,station_id,hour_dt,num_docks_available,year,month,day,hour,ctx_4,ctx_3,ctx_2,ctx_1,row_idx
4,2,2021-10-01 00:00:00,28.000000,2021,10,1,0,1.638471e+09,24.000000,26.166667,27.583333,4
9,2,2021-10-01 05:00:00,9.083333,2021,10,1,5,2.800000e+01,28.000000,26.333333,10.583333,9
14,2,2021-10-01 10:00:00,24.500000,2021,10,1,10,8.666667e+00,20.500000,26.000000,25.083333,14
19,2,2021-10-01 15:00:00,21.166667,2021,10,1,15,1.983333e+01,8.583333,13.333333,15.916667,19
24,2,2021-10-01 20:00:00,25.416667,2021,10,1,20,2.433333e+01,26.500000,27.416667,25.500000,24
29,2,2021-10-02 01:00:00,27.166667,2021,10,2,1,2.583333e+01,25.416667,24.000000,24.833333,29
34,2,2021-10-02 06:00:00,15.000000,2021,10,2,6,2.600000e+01,24.416667,14.500000,14.166667,34
39,2,2021-10-02 11:00:00,25.083333,2021,10,2,11,1.566667e+01,20.333333,21.666667,23.333333,39
44,2,2021-10-02 16:00:00,24.166667,2021,10,2,16,2.366667e+01,15.250000,22.166667,25.500000,44
49,2,2021-10-02 21:00:00,26.666667,2021,10,2,21,2.466667e+01,26.250000,26.166667,25.416667,49


### Processing 2020

In [123]:
ddf_2020_Q1 = load_quarter(2020, "Q1", files_by_year_quarter)
ddf_final_subset_2020_q1 = process_ddf_quarter(ddf_2020_Q1)

ddf_2020_Q2 = load_quarter(2020, "Q2", files_by_year_quarter)
ddf_final_subset_2020_q2 = process_ddf_quarter(ddf_2020_Q2)

ddf_2020_Q3 = load_quarter(2020, "Q3", files_by_year_quarter)
ddf_final_subset_2020_q3 = process_ddf_quarter(ddf_2020_Q3)

ddf_2020_Q4 = load_quarter(2020, "Q4", files_by_year_quarter)
ddf_final_subset_2020_q4 = process_ddf_quarter(ddf_2020_Q4)

####  Cecking... NO-filtered data processed

In [124]:
print(len(ddf_final_subset_2020_q1))
ddf_final_subset_2020_q1.head(10)

90461


,station_id,hour_dt,num_docks_available,year,month,day,hour,ctx_4,ctx_3,ctx_2,ctx_1,row_idx
4,1,2020-02-01 02:00:00,8.250000,2020,2,1,2,17.000000,14.416667,12.500000,11.083333,4
9,1,2020-02-01 07:00:00,12.333333,2020,2,1,7,8.000000,7.166667,8.181818,9.307692,9
14,1,2020-02-01 12:00:00,32.583333,2020,2,1,12,14.750000,16.916667,23.083333,29.750000,14
19,1,2020-02-01 17:00:00,40.416667,2020,2,1,17,41.500000,42.083333,41.250000,41.666667,19
24,1,2020-02-01 22:00:00,38.583333,2020,2,1,22,38.916667,36.416667,36.333333,39.750000,24
29,1,2020-02-02 03:00:00,29.166667,2020,2,2,3,35.750000,32.083333,30.416667,29.833333,29
34,1,2020-02-02 08:00:00,36.166667,2020,2,2,8,29.250000,29.000000,29.583333,32.250000,34
39,1,2020-02-02 13:00:00,30.750000,2020,2,2,13,38.250000,40.333333,42.833333,36.750000,39
44,1,2020-02-02 18:00:00,21.750000,2020,2,2,18,40.166667,43.250000,24.000000,19.583333,44
49,1,2020-02-02 23:00:00,7.181818,2020,2,2,23,21.250000,17.166667,15.416667,11.500000,49


In [125]:
print(len(ddf_final_subset_2020_q2))
ddf_final_subset_2020_q2.head(10)

151438


,station_id,hour_dt,num_docks_available,year,month,day,hour,ctx_4,ctx_3,ctx_2,ctx_1,row_idx
4,1,2020-04-23 02:00:00,44.000000,2020,4,23,2,46.000000,46.000000,45.000000,44.000000,4
9,1,2020-04-23 07:00:00,25.666667,2020,4,23,7,44.000000,44.000000,44.000000,29.083333,9
14,1,2020-04-23 12:00:00,23.333333,2020,4,23,12,24.416667,24.000000,24.000000,24.000000,14
19,1,2020-04-23 17:00:00,25.000000,2020,4,23,17,23.666667,24.000000,24.166667,25.083333,19
24,1,2020-04-23 22:00:00,23.000000,2020,4,23,22,25.000000,23.846154,23.000000,23.000000,24
29,1,2020-04-24 03:00:00,23.000000,2020,4,24,3,23.000000,23.000000,23.000000,23.000000,29
34,1,2020-04-24 08:00:00,23.230769,2020,4,24,8,23.083333,24.416667,25.000000,24.000000,34
39,1,2020-04-24 13:00:00,24.250000,2020,4,24,13,16.583333,19.750000,19.250000,21.166667,39
44,1,2020-04-24 18:00:00,24.000000,2020,4,24,18,23.750000,23.000000,23.833333,24.000000,44
49,1,2020-04-24 23:00:00,24.000000,2020,4,24,23,24.000000,24.000000,24.000000,24.000000,49


In [126]:
print(len(ddf_final_subset_2020_q3))
ddf_final_subset_2020_q3.head(10)

207572


,station_id,hour_dt,num_docks_available,year,month,day,hour,ctx_4,ctx_3,ctx_2,ctx_1,row_idx
4,2,2020-07-01 01:00:00,16.583333,2020,7,1,1,20.000000,18.416667,17.416667,16.000000,4
9,2,2020-07-01 06:00:00,19.666667,2020,7,1,6,17.833333,18.000000,18.083333,15.500000,9
14,2,2020-07-01 11:00:00,24.666667,2020,7,1,11,21.833333,23.666667,23.333333,23.916667,14
19,2,2020-07-01 16:00:00,23.166667,2020,7,1,16,22.000000,19.750000,18.750000,19.416667,19
24,2,2020-07-01 21:00:00,13.833333,2020,7,1,21,23.333333,18.833333,11.583333,11.083333,24
29,2,2020-07-02 02:00:00,12.000000,2020,7,2,2,14.500000,13.916667,14.166667,12.750000,29
34,2,2020-07-02 07:00:00,13.666667,2020,7,2,7,12.000000,11.500000,10.666667,9.833333,34
39,2,2020-07-02 12:00:00,4.333333,2020,7,2,12,14.916667,16.500000,13.166667,6.750000,39
44,2,2020-07-02 17:00:00,20.166667,2020,7,2,17,4.083333,6.250000,16.416667,20.416667,44
49,2,2020-07-02 22:00:00,9.500000,2020,7,2,22,21.583333,18.666667,13.416667,13.166667,49


In [127]:
print(len(ddf_final_subset_2020_q4))
ddf_final_subset_2020_q4.head(10)

ValueError: Mismatched dtypes found in `pd.read_csv`/`pd.read_table`.

+--------------+--------+----------+
| Column       | Found  | Expected |
+--------------+--------+----------+
| is_returning | object | float64  |
+--------------+--------+----------+

The following columns also raised exceptions on conversion:

- is_returning
  ValueError("could not convert string to float: 'IN_SERVICE'")

Usually this is due to dask's dtype inference failing, and
*may* be fixed by specifying dtypes manually by adding:

dtype={'is_returning': 'object'}

to the call to `read_csv`/`read_table`.

#### Cecking... Filtered data processed

In [128]:
ddf_final_subset_2020_q1_filtered = ddf_final_subset_2020_q1[ddf_final_subset_2020_q1['station_id'].isin(valid_station_ids)]
print(len(ddf_final_subset_2020_q1_filtered))
ddf_final_subset_2020_q1_filtered.head(10)

89176


,station_id,hour_dt,num_docks_available,year,month,day,hour,ctx_4,ctx_3,ctx_2,ctx_1,row_idx
4,1,2020-02-01 02:00:00,8.250000,2020,2,1,2,17.000000,14.416667,12.500000,11.083333,4
9,1,2020-02-01 07:00:00,12.333333,2020,2,1,7,8.000000,7.166667,8.181818,9.307692,9
14,1,2020-02-01 12:00:00,32.583333,2020,2,1,12,14.750000,16.916667,23.083333,29.750000,14
19,1,2020-02-01 17:00:00,40.416667,2020,2,1,17,41.500000,42.083333,41.250000,41.666667,19
24,1,2020-02-01 22:00:00,38.583333,2020,2,1,22,38.916667,36.416667,36.333333,39.750000,24
29,1,2020-02-02 03:00:00,29.166667,2020,2,2,3,35.750000,32.083333,30.416667,29.833333,29
34,1,2020-02-02 08:00:00,36.166667,2020,2,2,8,29.250000,29.000000,29.583333,32.250000,34
39,1,2020-02-02 13:00:00,30.750000,2020,2,2,13,38.250000,40.333333,42.833333,36.750000,39
44,1,2020-02-02 18:00:00,21.750000,2020,2,2,18,40.166667,43.250000,24.000000,19.583333,44
49,1,2020-02-02 23:00:00,7.181818,2020,2,2,23,21.250000,17.166667,15.416667,11.500000,49


In [129]:
ddf_final_subset_2020_q2_filtered = ddf_final_subset_2020_q2[ddf_final_subset_2020_q2['station_id'].isin(valid_station_ids)]
print(len(ddf_final_subset_2020_q2_filtered))
ddf_final_subset_2020_q2_filtered.head(10)

149506


,station_id,hour_dt,num_docks_available,year,month,day,hour,ctx_4,ctx_3,ctx_2,ctx_1,row_idx
4,1,2020-04-23 02:00:00,44.000000,2020,4,23,2,46.000000,46.000000,45.000000,44.000000,4
9,1,2020-04-23 07:00:00,25.666667,2020,4,23,7,44.000000,44.000000,44.000000,29.083333,9
14,1,2020-04-23 12:00:00,23.333333,2020,4,23,12,24.416667,24.000000,24.000000,24.000000,14
19,1,2020-04-23 17:00:00,25.000000,2020,4,23,17,23.666667,24.000000,24.166667,25.083333,19
24,1,2020-04-23 22:00:00,23.000000,2020,4,23,22,25.000000,23.846154,23.000000,23.000000,24
29,1,2020-04-24 03:00:00,23.000000,2020,4,24,3,23.000000,23.000000,23.000000,23.000000,29
34,1,2020-04-24 08:00:00,23.230769,2020,4,24,8,23.083333,24.416667,25.000000,24.000000,34
39,1,2020-04-24 13:00:00,24.250000,2020,4,24,13,16.583333,19.750000,19.250000,21.166667,39
44,1,2020-04-24 18:00:00,24.000000,2020,4,24,18,23.750000,23.000000,23.833333,24.000000,44
49,1,2020-04-24 23:00:00,24.000000,2020,4,24,23,24.000000,24.000000,24.000000,24.000000,49


In [130]:
ddf_final_subset_2020_q3_filtered = ddf_final_subset_2020_q3[ddf_final_subset_2020_q3['station_id'].isin(valid_station_ids)]
print(len(ddf_final_subset_2020_q3_filtered))
ddf_final_subset_2020_q3_filtered.head(10)

205411


,station_id,hour_dt,num_docks_available,year,month,day,hour,ctx_4,ctx_3,ctx_2,ctx_1,row_idx
4,2,2020-07-01 01:00:00,16.583333,2020,7,1,1,20.000000,18.416667,17.416667,16.000000,4
9,2,2020-07-01 06:00:00,19.666667,2020,7,1,6,17.833333,18.000000,18.083333,15.500000,9
14,2,2020-07-01 11:00:00,24.666667,2020,7,1,11,21.833333,23.666667,23.333333,23.916667,14
19,2,2020-07-01 16:00:00,23.166667,2020,7,1,16,22.000000,19.750000,18.750000,19.416667,19
24,2,2020-07-01 21:00:00,13.833333,2020,7,1,21,23.333333,18.833333,11.583333,11.083333,24
29,2,2020-07-02 02:00:00,12.000000,2020,7,2,2,14.500000,13.916667,14.166667,12.750000,29
34,2,2020-07-02 07:00:00,13.666667,2020,7,2,7,12.000000,11.500000,10.666667,9.833333,34
39,2,2020-07-02 12:00:00,4.333333,2020,7,2,12,14.916667,16.500000,13.166667,6.750000,39
44,2,2020-07-02 17:00:00,20.166667,2020,7,2,17,4.083333,6.250000,16.416667,20.416667,44
49,2,2020-07-02 22:00:00,9.500000,2020,7,2,22,21.583333,18.666667,13.416667,13.166667,49


In [131]:
ddf_final_subset_2020_q4_filtered = ddf_final_subset_2020_q4[ddf_final_subset_2020_q4['station_id'].isin(valid_station_ids)]
print(len(ddf_final_subset_2020_q4_filtered))
ddf_final_subset_2020_q4_filtered.head(10)

ValueError: Mismatched dtypes found in `pd.read_csv`/`pd.read_table`.

+--------------+--------+----------+
| Column       | Found  | Expected |
+--------------+--------+----------+
| is_returning | object | float64  |
+--------------+--------+----------+

The following columns also raised exceptions on conversion:

- is_returning
  ValueError("could not convert string to float: 'IN_SERVICE'")

Usually this is due to dask's dtype inference failing, and
*may* be fixed by specifying dtypes manually by adding:

dtype={'is_returning': 'object'}

to the call to `read_csv`/`read_table`.